In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
import math

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN
import time

tf.random.set_seed(42)

In [24]:
h_a = np.loadtxt('../../Data/data_39_lowSI.txt', dtype=float)

h_a = h_a[h_a>0]
h_a = h_a[1::20]

avr = np.average(h_a)
h_a = h_a/avr
h_a = h_a.reshape(-1,1)

In [25]:
N_train = 7500       # train len for each iteration
N_test = 2500      # test len
k = (len(h_a) - N_train) // N_test
N_total = N_train+k*N_test       # total sample len
out_size = 9
timesteps = 24

Y_test = np.zeros((N_total-N_train - k*(timesteps-1), out_size), dtype=float)
Y_pred = np.zeros((N_total-N_train - k*(timesteps-1), out_size), dtype=float)

In [26]:
model = Sequential([
    SimpleRNN(70, return_sequences=True, input_shape=(timesteps-out_size,1)),
    # Dropout(0.001),

    SimpleRNN(70, return_sequences=False),
    # Dropout(0.001),

    Dense(out_size),

])
model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 15, 70)            5040      
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 70)                9870      
                                                                 
 dense_2 (Dense)             (None, 9)                 639       
                                                                 
Total params: 15,549
Trainable params: 15,549
Non-trainable params: 0
_________________________________________________________________


In [27]:
for i in range(1):
    print('i = ' + str(i))

    train_data = h_a[i*N_test : N_train+i*N_test].reshape(-1,1)
    test_data = h_a[N_train + i*N_test : (i+1)*N_test + N_train].reshape(-1,1)

    train_data_timesteps=np.array([[j for j in train_data[i:i+timesteps]] for i in range(0,len(train_data)-timesteps+1)])[:,:,0]
    test_data_timesteps=np.array([[j for j in test_data[i:i+timesteps]] for i in range(0,len(test_data)-timesteps+1)])[:,:,0]

    x_train, y_train = train_data_timesteps[:,:timesteps-out_size],train_data_timesteps[:,timesteps-out_size:]
    x_test, y_test = test_data_timesteps[:,:timesteps-out_size],test_data_timesteps[:,timesteps-out_size:]

    print('Start training ............')
    st = time.time()
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
    history = model.fit(
        x_train,
        y_train,
        epochs = 150,
        batch_size = 32,
        verbose = 1,
    )
    et = time.time()
    print('Execution time = ' + str(et - st) + ' sec')

    # y_train_pred = model.predict(x_train).reshape(-1,1)
    print('Start predicting ............')
    st = time.time()
    y_test_pred = model.predict(x_test)
    et = time.time()
    print('Execution time = ' + str(et - st) + ' sec')
    
    Y_test[i*(N_test-timesteps+1):(i+1)*(N_test-timesteps+1), :] = y_test
    Y_pred[i*(N_test-timesteps+1):(i+1)*(N_test-timesteps+1), :] = y_test_pred
    print('Done! \n')

i = 0
Start training ............
Epoch 1/150
234/234 [==============================] - 5s 8ms/step - loss: 0.0541
Epoch 2/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0270
Epoch 3/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0254
Epoch 4/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0222
Epoch 5/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0228
Epoch 6/150
234/234 [==============================] - 2s 9ms/step - loss: 0.0205
Epoch 7/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0210
Epoch 8/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0194
Epoch 9/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0196
Epoch 10/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0205
Epoch 11/150
234/234 [==============================] - 2s 8ms/step - loss: 0.0193
Epoch 12/150
234/234 [==============================] - 2s 8ms

In [28]:
for i in range(1,k):
    print('i = ' + str(i))

    train_data = h_a[i*N_test : N_train+i*N_test].reshape(-1,1)
    test_data = h_a[N_train + i*N_test : (i+1)*N_test + N_train].reshape(-1,1)

    train_data_timesteps=np.array([[j for j in train_data[i:i+timesteps]] for i in range(0,len(train_data)-timesteps+1)])[:,:,0]
    test_data_timesteps=np.array([[j for j in test_data[i:i+timesteps]] for i in range(0,len(test_data)-timesteps+1)])[:,:,0]

    x_train, y_train = train_data_timesteps[:,:timesteps-out_size],train_data_timesteps[:,timesteps-out_size:]
    x_test, y_test = test_data_timesteps[:,:timesteps-out_size],test_data_timesteps[:,timesteps-out_size:]

    print('Start training ............')
    st = time.time()
    early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
    history = model.fit(
        x_train,
        y_train,
        epochs = 5,
        batch_size = 32,
        verbose = 1,
    )
    et = time.time()
    print('Execution time = ' + str(et - st) + ' sec')

    # y_train_pred = model.predict(x_train).reshape(-1,1)
    print('Start predicting ............')
    st = time.time()
    y_test_pred = model.predict(x_test)
    et = time.time()
    print('Execution time = ' + str(et - st) + ' sec')
    
    Y_test[i*(N_test-timesteps+1):(i+1)*(N_test-timesteps+1), :] = y_test
    Y_pred[i*(N_test-timesteps+1):(i+1)*(N_test-timesteps+1), :] = y_test_pred
    print('Done! \n')

i = 1


Start training ............
Epoch 1/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0164
Epoch 2/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0162
Epoch 3/5
234/234 [==============================] - 2s 7ms/step - loss: 0.0165
Epoch 4/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0164
Epoch 5/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0160
Execution time = 9.380169868469238 sec
Start predicting ............
78/78 [==============================] - 0s 4ms/step
Execution time = 0.519676923751831 sec
Done! 

i = 2
Start training ............
Epoch 1/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0183
Epoch 2/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0183
Epoch 3/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0182
Epoch 4/5
234/234 [==============================] - 2s 8ms/step - loss: 0.0181
Epoch 5/5
234/234 [==============================

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import math

MSE = mean_squared_error(Y_test, Y_pred)
RMSE = math.sqrt(MSE)
MAE = mean_absolute_error(Y_test, Y_pred)
MAPE = mean_absolute_percentage_error(Y_test, Y_pred)

print('MSE = ', MSE)
print('RMSE = ', RMSE)
print('MAE = ', MAE)
print('MAPE = ', MAPE)

MSE =  0.0006498368986517302
RMSE =  0.025491898686675542
MAE =  0.01766036845870753
MAPE =  0.017741237303100208


In [30]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import math

MSE = [0 for j in range(out_size)]
RMSE = [0 for j in range(out_size)]
MAE = [0 for j in range(out_size)]
MAPE = [0 for j in range(out_size)]

for i in range(out_size):
    MSE[i] = mean_squared_error(Y_test[:,i], Y_pred[:,i])
    RMSE[i] = math.sqrt(MSE[i])
    MAE[i] = mean_absolute_error(Y_test[:,i], Y_pred[:,i])
    MAPE[i] = mean_absolute_percentage_error(Y_test[:,i], Y_pred[:,i])

print('MSE = ', MSE)
print('RMSE = ', RMSE)
print('MAE = ', MAE)
print('MAPE = ', MAPE)

MSE =  [6.53243068092009e-05, 0.00011400252892612699, 0.00020961988688774334, 0.0003382472931600083, 0.0005202317604898932, 0.0007351172912032748, 0.000994248418800281, 0.0012821169014136187, 0.0015896237001754461]
RMSE =  [0.008082345377995234, 0.010677196679190985, 0.014478255657631665, 0.018391500568469346, 0.022808589620796224, 0.02711304651276346, 0.031531704977693184, 0.03580666001477405, 0.03987008527925976]
MAE =  [0.006329023398831557, 0.008288064007195142, 0.011094079940417705, 0.014039960829195652, 0.017329843595064554, 0.020580950992595687, 0.023911558414127457, 0.027154276416141855, 0.030215558534829212]
MAPE =  [0.00635472595899905, 0.008323265647913932, 0.011142041449878813, 0.014101418527777708, 0.017404362340297453, 0.020672197011574068, 0.024022902049818363, 0.027279733749348476, 0.030370488992296323]


In [33]:
np.save("../Results/Multi_step/lowSI/RNN_test", Y_test)
np.save("../Results/Multi_step/lowSI/RNN_pred", Y_pred)

In [34]:
MAE = np.zeros((out_size))

for j in range(out_size):
    Y_t = Y_test[:,j]
    Y_p = Y_pred[:,j]

    MAE[j] = mean_absolute_error(Y_t, Y_p)

    np.savetxt("../../Data/Multi-step-RNN/lowSI_39/real_" + str(j)+ ".txt", Y_t, delimiter = '\n')
    np.savetxt("../../Data/Multi-step-RNN/lowSI_39/predicted_" + str(j)+ ".txt", Y_p, delimiter = '\n')

print(MAE)
np.savetxt("../../Data/Multi-step-RNN/lowSI_39/MAE.txt", MAE, delimiter='\n')

[0.00632902 0.00828806 0.01109408 0.01403996 0.01732984 0.02058095
 0.02391156 0.02715428 0.03021556]
